In [1]:
import logging
import numpy as np
import xarray as xr
from distributed import Client

from nwsspc.sharp.calc import constants
from nwsspc.sharp.calc import parcel
from nwsspc.sharp.calc import params
from nwsspc.sharp.calc import thermo
from nwsspc.sharp.calc import layer

client = Client(n_workers=8, silence_logs=logging.ERROR)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 62.32 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43989,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:42237,Total threads: 2
Dashboard: http://127.0.0.1:46445/status,Memory: 7.79 GiB
Nanny: tcp://127.0.0.1:44071,


In [ ]:
ds_hybrid = xr.open_dataset("hrrr-hybrid.json", engine="kerchunk", decode_timedelta=True)[["pres", "gh", "t", "q"]].chunk(dict(hybrid=-1, y=1, x=1799)).load()
ds_2m = xr.open_dataset("hrrr-2m.json", engine="kerchunk", decode_timedelta=True)[["t2m", "sh2"]].chunk(dict(y=1, x=1799)).load()
ds_sfc = xr.open_dataset("hrrr-surface.json", engine="kerchunk", decode_timedelta=True)[["sp"]].chunk(dict( y=1, x=1799)).load()

In [ ]:
print(ds_hybrid)

In [ ]:
def compute_everything(pres, hght, tmpk, spfh, sp, t2m, sh2, use_2m=True):
    mixr = thermo.mixratio(spfh)
    mixr_2m = thermo.mixratio(sh2)

    mixr[mixr < constants.TOL] = constants.TOL
    if (mixr_2m < constants.TOL): mixr_2m = constants.TOL

    vtmp = thermo.virtual_temperature(tmpk, mixr)

    if (use_2m):
        dwpk_2m = thermo.temperature_at_mixratio(mixr_2m, sp)
        pcl = parcel.Parcel(sp, t2m, dwpk_2m, parcel.LPL.SFC)
    else: 
        pcl_dwpk = thermo.temperature_at_mixratio(mixr[0], pres[0])
        pcl = parcel.Parcel(pres[0], tmpk[0], pcl_dwpk, parcel.LPL.SFC)

    lifter = parcel.lifter_cm1()
    lifter.ma_type = thermo.adiabat.pseudo_liq
    lifter.converge = 0.1

    pcl_vtmp = pcl.lift_parcel(lifter, pres)
    pcl_buoy = thermo.buoyancy(pcl_vtmp, vtmp)
    cape, cinh = pcl.cape_cinh(pres, hght, pcl_buoy)
    return cape

In [ ]:
input_core_dims = [
    ["hybrid"], ["hybrid"],
    ["hybrid"], ["hybrid"],
    [], [], []
]



In [ ]:
cape_2m = xr.apply_ufunc(
    compute_everything, 
    ds_hybrid["pres"],
    ds_hybrid["gh"],
    ds_hybrid["t"], 
    ds_hybrid["q"],
    ds_sfc["sp"],
    ds_2m["t2m"],
    ds_2m["sh2"],
    input_core_dims=input_core_dims,
    output_core_dims=[[]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[float],
    kwargs={"use_2m": False},
).compute()

print(cape_2m.max())